In [2]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras  import layers
from keras_tuner.tuners import RandomSearch

In [3]:
df=pd.read_csv("C:\\Users\\Kedar-PC\\Desktop\\Dataset\\real_combine.csv")

In [4]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [6]:
df.shape

(1093, 9)

In [7]:
df.isnull().sum()

T         0
TM        0
Tm        0
SLP       0
H         0
VV        0
V         0
VM        0
PM 2.5    1
dtype: int64

In [17]:
df['PM 2.5']=df['PM 2.5'].fillna(df['PM 2.5'].mean())

In [19]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1093 entries, 0 to 1092
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   T       1093 non-null   float64
 1   TM      1093 non-null   float64
 2   Tm      1093 non-null   float64
 3   SLP     1093 non-null   float64
 4   H       1093 non-null   float64
 5   VV      1093 non-null   float64
 6   V       1093 non-null   float64
 7   VM      1093 non-null   float64
 8   PM 2.5  1093 non-null   float64
dtypes: float64(9)
memory usage: 77.0 KB


In [21]:
x=df.drop('PM 2.5',axis=True)

In [22]:
y=df['PM 2.5']

In [23]:
x.head()

,T,TM,Tm,SLP,H,VV,V,VM
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2


In [24]:
y.head()

0    219.720833
1    182.187500
2    154.037500
3    223.208333
4    200.645833
Name: PM 2.5, dtype: float64

In [25]:
def build_model(hp):
    model=keras.Sequential()
    for i in range(hp.Int('num_layers',2,20)):
        model.add(layers.Dense(units=hp.Int('units_'+ str(i),min_value=32,max_value=512,step=32),activation='relu'))
        model.add(layers.Dense(1,activation='linear'))
        model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',[1e-2,1e-4,1e-6])),loss='mean_absolute_error',
                     metrics=['mean_absolute_error'])
        return model

In [26]:
tuner=RandomSearch(build_model,objective='val_mean_absolute_error',max_trials=5,
                  executions_per_trial=3,directory='project1',project_name='AI Quality Index')

In [27]:
tuner.search_space_summary()

Search space summary
Default search space size: 3
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.0001, 1e-06], 'ordered': True}


In [28]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=45)

In [30]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 06s]
val_mean_absolute_error: 50.09646733601888

Best val_mean_absolute_error So Far: 49.40623474121094
Total elapsed time: 00h 00m 39s
INFO:tensorflow:Oracle triggered exit


In [31]:
tuner.results_summary()

Results summary
Results in project1\AI Quality Index
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 20
units_0: 352
learning_rate: 0.01
Score: 49.40623474121094
Trial summary
Hyperparameters:
num_layers: 15
units_0: 192
learning_rate: 0.01
Score: 49.939474741617836
Trial summary
Hyperparameters:
num_layers: 3
units_0: 480
learning_rate: 0.01
Score: 50.09646733601888
Trial summary
Hyperparameters:
num_layers: 9
units_0: 128
learning_rate: 1e-06
Score: 139.4849090576172
Trial summary
Hyperparameters:
num_layers: 5
units_0: 160
learning_rate: 1e-06
Score: 158.7527313232422
